In [1]:
import os


In [2]:
%pwd

'd:\\ML\\potato_disease_predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\potato_disease_predictor'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from Potato_disease_predictor_.constants import *
from Potato_disease_predictor_.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


    

In [8]:
import os
import urllib.request as request
import zipfile
from Potato_disease_predictor_ import logger
from Potato_disease_predictor_.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(url=self.config.source_URL,
                                                   filename=self.config.local_data_file)
            logger.info(f"{filename} download! with following info :\n{headers}")

        else:
            
            logger.info(f"file already exist of size : {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
               zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile:
          print(f"Error: The file '{self.config.local_data_file}' is not a valid ZIP file or is corrupted.")
        except FileNotFoundError:
          print(f"Error: The file '{self.config.local_data_file}' does not exist.")
            





In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2024-10-23 13:28:08,318: INFO :common: yaml file : config\config.yaml loaded successfully]
[2024-10-23 13:28:08,320: INFO :common: yaml file : params.yaml loaded successfully]
[2024-10-23 13:28:08,323: INFO :common: created directory at : artifacts]
[2024-10-23 13:28:08,326: INFO :common: created directory at : artifacts/data_ingestion]
[2024-10-23 13:28:08,328: INFO :2695903690: file already exist of size : ~38866 KB]
